In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
import torch.distributions as tdist
import sys
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d, BatchNorm2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from scipy.stats import norm
# matplotlib options
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
#Read in the pre-processed data
df = pd.read_csv('prepared/demand_supply/10244.csv')
#Baseline
df.time = pd.to_datetime(df.time)
demand_ts = [df.demand[t] for t in range(len(df))]
supply_ts = [df.supply[t] for t in range(len(df))]
print("Fraction of boundary cases: %.3f" % (np.sum(np.array(demand_ts) == np.array(supply_ts)) / len(demand_ts)),)
print("Fraction of problematic cases: %.3f" % (np.sum(np.array(demand_ts) > np.array(supply_ts)) / len(demand_ts)),)
plt.plot(demand_ts[:(24*7)], "r-")
plt.plot(supply_ts[:(24*7)], "b-")
plt.legend(["demand", "supply"], loc="upper left")
plt.show()

from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

plot_acf(df.demand, lags=48)
plot_pacf(df.demand, lags=24)
plt.show()

In [ ]:
## Add more data from other zones:

df2 = pd.read_csv('prepared/demand_supply/10222.csv')
df2.time = pd.to_datetime(df.time)

df3 = pd.read_csv('prepared/demand_supply/10234.csv')
df3.time = pd.to_datetime(df.time)

df4 = pd.read_csv('prepared/demand_supply/17304.csv')
df4.time = pd.to_datetime(df.time)

df = pd.concat([df,df2,df3,df4], axis = 0).reset_index(drop=True)
df.time = pd.to_datetime(df.time)



In [ ]:
for i in range(df.shape[0]):
    if df.demand[i] > df.supply[i]:
        df.supply[i] = df.demand[i]
        
dim = df.shape[0]

ToD = [df.time[i].hour for i in range(dim)]
DoW = [df.time[i].weekday() for i in range(dim)]
DoM = [df.time[i].day for i in range(dim)]
Month = [df.time[i].month for i in range(dim)]
df['ToD'] = ToD
df['DoW'] = DoW
df['DoM'] = DoM
df['Month'] = Month

df = df[['Month','DoM','DoW','ToD','demand','supply']].sort_values(['Month', 'DoM','DoW','ToD']).reset_index(drop=True)
Days = ['Mon', 'Tue','Wed','Thu','Fri','Sat','Sun']
#df.DoW = [Days[each] for each in df.DoW]
#Converting to categorical - the month and date is excluded for now
df = pd.concat([df, pd.get_dummies(df.DoW), pd.get_dummies(df.ToD)], axis = 1)
df.head(20)

In [ ]:
X = df.iloc[:,6:]
y = df.demand
z = df.supply

#Prepare data for training
X = np.asarray(X)
y = np.asarray(y)
z = np.asarray(z)

  
X_lag = np.transpose(np.asmatrix([np.roll(y,1), np.roll(y,2), np.roll(y,3), np.roll(y,4), np.roll(y,5), np.roll(y,6), np.roll(y,7), np.roll(y,8), np.roll(y,9), np.roll(y,10), np.roll(y,11), np.roll(y,12), np.roll(y,13), np.roll(y,14), np.roll(y,15), np.roll(y,16), np.roll(y,17), np.roll(y,18), np.roll(y,19), np.roll(y,20), np.roll(y,21), np.roll(y,22), np.roll(y,23), np.roll(y,24)]))
X_lag = np.expand_dims(X_lag, axis = 2)

obs, time_steps, lag_dim = X_lag.shape
num_features = X.shape[1]

train_idx, val_idx = int(obs*0.8), int(obs*0.9)

x_train = X[:train_idx,:].astype('float32')
x_train_lag = X_lag[:train_idx,:].astype('float32')
targets_train = y[:train_idx].astype('float32')
supply_train = z[:train_idx].astype('float32')

x_valid = X[train_idx:val_idx,:].astype('float32')
x_valid_lag = X_lag[train_idx:val_idx,:].astype('float32')
targets_valid = y[train_idx:val_idx].astype('float32')
supply_valid = z[train_idx:val_idx].astype('float32')

x_test = X[val_idx:,:].astype('float32')
x_test_lag = X_lag[val_idx:,:].astype('float32')
targets_test = y[val_idx:].astype('float32')
supply_test = z[val_idx:].astype('float32')

In [ ]:
use_cuda = torch.cuda.is_available()

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [ ]:
use_cuda = False
batch_size = 128
hidden_rnn = 100
hidden_FF = 126

In [ ]:
class Net(nn.Module):

    def __init__(self, seq_len, hidden_rnn):
        super().__init__()
        
        self.num_layers = 1
        self.hidden_rnn = hidden_rnn
        
        self.rnn = nn.LSTM(lag_dim, hidden_rnn, self.num_layers, batch_first = True)
        self.rnn_dense = nn.Linear(in_features = self.hidden_rnn, out_features = self.hidden_rnn)
        self.rnn_dense2 = nn.Linear(in_features = self.hidden_rnn, out_features = 1)
        
        
        self.dropout = Dropout(0.3)
      
        
        
    def forward(self, time_series, h0, c0):
        # Input shape [batch, seq_in_len, num_layers]
        
        output_rnn, (hn, cn) = self.rnn(time_series, (h0, c0))
        out_rnn = self.dropout(self.rnn_dense(hn[-1]))
        out_rnn = self.rnn_dense2(out_rnn)     
        
        return out_rnn

    def init_hidden(self, batch_size):
        init = Variable(torch.randn(self.num_layers, batch_size, self.hidden_rnn)).float()
        return init
    def init_cell(self, batch_size):
        init = Variable(torch.randn(self.num_layers, batch_size, self.hidden_rnn)).float()
        return init
    
net = Net(time_steps, hidden_rnn)
if use_cuda:
    net.cuda()
print(net)

In [ ]:
#Test
print(x_train.shape)
x_lag = get_variable(Variable(torch.from_numpy(x_train_lag[:batch_size,:]).float()))
print(x_lag.size())
h0 = net.init_hidden(batch_size)
c0 = net.init_cell(batch_size)
output = net(x_lag, c0, h0)
output

In [ ]:
import math
def standard_normal_pdf(x):
    
    return (1/math.sqrt(2 * math.pi))*torch.exp(-x**2/2)
    

def standard_normal_cdf(x):
    return 0.5 * (1 + torch.erf(x / math.sqrt(2)))


def censored_NLL(output, labels, supply):
    x = torch.squeeze(output)
    di = get_numpy((labels == supply).float())
    loss = get_variable(Variable(torch.zeros(1)))
    for i in range(len(output)):
        if di[i] == 0:
            if get_numpy(standard_normal_pdf(labels[i] - x[i])) == 0:
                loss += 20
            else:
                loss += -torch.log(standard_normal_pdf(labels[i]-x[i]))
        if di[i] == 1:
            if get_numpy(standard_normal_cdf(labels[i] - x[i])) == 1:
                loss += 20
            else:
                loss += -torch.log(1-standard_normal_cdf(labels[i] - x[i]))
             
    return loss



In [ ]:
def censored_NLL(output, labels, supply):
    x = torch.squeeze(output)
    di = get_numpy((labels == supply).float())
    loss = get_variable(Variable(torch.zeros(1)))
    for i in range(len(output)):
        if di[i] == 0:
            loss += -torch.log(standard_normal_pdf((labels[i]-x[i])/torch.std(labels)))
        if di[i] == 1:
            loss += -torch.log(1-standard_normal_cdf((labels[i] - x[i])/torch.std(labels)))
             
    return loss

In [ ]:
#Good with MSE loss -> #
l_rate = 0.01; optimizer = torch.optim.Adam(net.parameters(), lr = l_rate, weight_decay = 0.01)
l_rate = 0.001; optimizer = torch.optim.Adam(net.parameters(), lr = l_rate, weight_decay = 0.01) #R^2 ~ 0.34

#l_rate = 0.005; optimizer = torch.optim.Adam(net.parameters(), lr = l_rate, weight_decay = 0.1)
#l_rate = 0.0001; optimizer = optim.SGD(net.parameters(), lr=l_rate, weight_decay = 0.1) #R^2 ~ 0.357
#l_rate = 0.0001; optimizer = torch.optim.RMSprop(net.parameters(), lr = l_rate, weight_decay = 0.01)

#criterion = censored_NLL
criterion = nn.MSELoss()
#epochs = 1

In [ ]:
epochs = 100

In [ ]:
from torch.nn.utils import clip_grad_norm


num_samples_train = x_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = x_valid.shape[0]
num_batches_valid = num_samples_valid // batch_size

train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
losses = []
losses_valid = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)
h0 = get_variable(net.init_hidden(batch_size))
c0 = get_variable(net.init_cell(batch_size))

# setting up lists for handling loss/accuracy

train_losses = []; valid_losses = [];
train_acc, valid_acc = [], []


for epoch in range(epochs):

    epoch +=1
    
    cur_loss = 0
    net.train()
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        x_batch = get_variable(Variable(torch.from_numpy(x_train[slce])).float())
        x_lag_batch = get_variable(Variable(torch.from_numpy(x_train_lag[slce])))
        supply_batch = get_variable(Variable(torch.from_numpy(supply_train[slce])).float())
        target_batch = get_variable(Variable(torch.from_numpy(targets_train[slce]).float()))
        
        output = net(x_lag_batch, h0, c0)
        
        # compute gradients given loss
        target_batch = get_variable(Variable(torch.from_numpy(targets_train[slce]).float()))
       
        
        #batch_loss = criterion(output, target_batch, supply_batch)
        batch_loss = criterion(output, target_batch) # MSE Loss 
        
                
        #print(batch_loss)
        if np.isinf(get_numpy(batch_loss)):
            print("Error - loss is inf!")
            sys.exit() 
        if np.isnan(get_numpy(batch_loss)):
            print("Error - loss is Nan!")
            sys.exit() 
        
        
        optimizer.zero_grad()
        batch_loss.backward()
        
        #clip_grad_norm(net.parameters(), 10) #0.25 used in example
           
        
        optimizer.step()
        cur_loss += batch_loss
    losses.append(get_numpy(cur_loss / batch_size))
    
    
    
    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        x_batch = get_variable(Variable(torch.from_numpy(x_train[slce])).float())
        x_lag_batch = get_variable(Variable(torch.from_numpy(x_train_lag[slce])))
        output = net.forward(x_lag_batch, h0, c0);
        preds = get_numpy(output)
        
        train_targs += list(targets_train[slce])
        train_preds += list(preds)
    
    ### Evaluate validation
    val_loss = 0
    val_preds, val_targs = [], []
    for i in range(num_batches_valid):
        slce = get_slice(i, batch_size)
        x_val = get_variable(Variable(torch.from_numpy(x_valid[slce]).float()))
        x_lag_val = get_variable(Variable(torch.from_numpy(x_valid_lag[slce]).float()))
        supply_val = get_variable(Variable(torch.from_numpy(supply_valid[slce])).float())
        target_val = get_variable(Variable(torch.from_numpy(targets_valid[slce]).float()))
        
        output_valid = net.forward(x_lag_val, h0, c0)
        #valid_loss = criterion(output_valid, target_val, supply_val)
        valid_loss = criterion(output_valid, target_val)
        
        preds = get_numpy(output_valid)
        
        val_preds += list(preds)
        val_targs += list(targets_valid[slce])
        
        val_loss += valid_loss   
    losses_valid.append(get_numpy(val_loss / batch_size))

    train_acc_cur = r2_score(train_targs, train_preds)
    valid_acc_cur = r2_score(val_targs, val_preds)
    
    train_acc.append(train_acc_cur)
    valid_acc.append(valid_acc_cur)
    
    
    #predicted_valid = get_numpy(model.forward(inputs_val, h0, c0))
    #predicted_valid = predicted_valid[:,0,0]
    #valid_acc.append(r2_score(targets_val, predicted_val))
    
    
    
    print('epoch {}, loss {}, train accuracy {}, valid accuracy {}'.format(epoch,losses[epoch-1], train_acc[epoch-1], valid_acc[epoch-1]))
    #print('epoch {}, loss {}, train accuracy {}'.format(epoch,loss.data[0], train_acc[epoch-1]))